In [1]:
# 원본 텍스트 파일을 내려받아 파싱

import tensorflow as tf
import numpy as np
import random
import sys

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

path = keras.utils.get_file(
    'nietzsche.txt',
    origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('말뭉치 크기 : ', len(text))

# 글자 시퀀스 벡터화
maxlen = 120 # 60개의 글자로 된 시퀀스 추출
step = 3 # 세 글자씩 건너뛰면서 새로운 시퀀스를 샘플링

sentences = [] # 추출한 시퀀스를 담을 리스트
next_chars = [] # target(시퀀스 다음 글자)을 담을 리스트

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])

print('시퀀스 개수 : ', len(sentences))

chars = sorted(list(set(text))) # 말뭉치에서 고유한 글자를 담은 리스트
print('고유한 글자 : ', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars) # chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리

print('벡터화 진행중...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1  # 글자 one-hot encoding 하여 0과 1의 이진 배열로 나타냄
    
    
# 네트워크 구성
model = Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation = 'softmax'))

optimizer = tf.keras.optimizers.RMSprop(lr = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer) # 타겟이 원-핫 인코딩이므로 categorical 사용


말뭉치 크기 :  600893
시퀀스 개수 :  200258
고유한 글자 :  57
벡터화 진행중...


In [2]:
# 모델의 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수
'''
다항 분포 샘플링 함수인 multinomial(n, p, s)의 
n은 샘플링 횟수, p는 확률 분포 배열로 전체 합이 1이 되어야 한다. s는 샘플링 과정을 반복할 횟수이다.
n이 충분히 크면 반환되는 배열의 분포는 p의 분포에 가까워진다.
이 코드에서는 n을 1로 설정했기 때문에 (샘플링을 한 번만 수행하므로) 반환값은 하나의 원소만 1이고 나머지는 모두 0인 (s, len(p)) 크기의 배열이다.
'''
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# 텍스트 생성 루프

random.seed(42)
start_index = random.randint(0, len(text) - maxlen -1)

for epoch in range(1, 60):  # 60 epoch 동안 모델을 훈련
    print('\n\n에포크 ', epoch)
    model.fit(x, y, batch_size = 128, epochs = 1)  # 데이터에서 한 번만 반복해서 모델을 학습
    
    seed_text = text[start_index : start_index + maxlen]  # 무작위 시드 텍스트 선택
    print('\n시드 텍스트 : ', seed_text)
    
    for temperature in [0.3, 0.5, 0.7, 1.0]:
        print("\nTemperature 값 : ", temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400):  # 시드 텍스트에서 시작해 400개의 글자 생성
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.  # 지금까지 생성된 글자를 one-hot encoding
                
            preds = model.predict(sampled, verbose = 0)[0]  # 다음 글자 샘플링
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
            
        print()



에포크  1
1565/1565 [==============================] - 18s 12ms/step - loss: 1.9578

시드 텍스트 :  the slowly ascending ranks and classes, in which,
through fortunate marriage customs, volitional power and delight in
se

Temperature 값 :  0.3
the slowly ascending ranks and classes, in which,
through fortunate marriage customs, volitional power and delight in
seestion of the compless of the conscience of the precious of the comely all the soul--and the sense of the sense the some and religious and feeling who however of the comence of the soul of the enest in the conscience is a more of the precisely to and man and to the conscience and of it is the postress of the sense of the formen to the believe of the conscience of the comence of the postent of all 

Temperature 값 :  0.5
the slowly ascending ranks and classes, in which,
through fortunate marriage customs, volitional power and delight in
sensition of his one ancestare a periliss of the will to all everything o

In [1]:
# 다른 온도 값을 사용하여 확률 분포의 가중치 바꾸기
'''
소프트맥스 온도(Softmax Temperature) : 이 파라미터는 샘플링에 사용되는 확률 분포의 엔트로피를 나타낸다. temperature 값이 주어지면 다음과 
같이 주어지면 다음과 같이 가중치를 적용하여 (모델의 소프트맥스 출력인) 원본 확률 분포에서 새로운 확률 분포를 계산한다.

이 함수는 1보다 작은 실수 배열(original_distribution)을 로그 스케일로 바꾸어 모두 음수로 만든다.
temperature가 1보다 작으면 더 큰 음수가 되므로 원본 스케일로 복원했을 때 배열 원소 간의 차이가 더 커진다. 반대로 1보다 클수록 0에 가까워지고
원본 스케일로 복원했을 때 배열 원소가 비슷한 값이 된다.

temperature 값이 낮아지면 다른 확률은 거의 0에 가까워져 탐욕적 샘플링(greedy sampling)이 된다.
'''
import tensorflow as tf
import os
import numpy as np

def reweight_distribution(original_distribution, temperature = 0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('텍스트의 길이: {}자'.format(len(text)))

1122304/1115394 [==============================] - 0s 0us/step
텍스트의 길이: 1115394자
